In [1]:
import pandas as pd
#import seaborn as sns
import numpy as np
#import matplotlib.pyplot as plt
import glob
from netCDF4 import Dataset
import json
import xarray as xr
#%matplotlib inline

In [2]:
save_dir = '/home/hanna/images_simple_statsmodels/'

In [3]:
coord_dir = '/home/hanna/Desktop/examples_master_thesis/'

In [4]:
coords_fil = glob.glob(coord_dir + '*.nc')[0]

In [5]:
coords_fil

'/home/hanna/Desktop/examples_master_thesis/W_XX-EUMETSAT-Darmstadt,SING+LEV+SAT,MSG4+CLM_C_EUMG_20190612113000.nc'

In [6]:
lat_lon = Dataset(coords_fil)

In [7]:
lat_array = lat_lon.variables['lat']
lon_array = lat_lon.variables['lon']

In [8]:
nc_files = glob.glob(save_dir+'*cell*.json')

In [9]:
nc_files

['/home/hanna/images_simple_statsmodels/changes_lat_lon_cell-area.json']

In [10]:
def add(a, b):
    return np.abs(a) + np.abs(b)

def subtract(a, b):
    return np.abs(a) - np.abs(b)

In [11]:
def read_dlon_dlat(save_dir):
    nc_files = glob.glob(save_dir+'*cell*.json')
    print(nc_files)
    
    with open(nc_files[-1]) as f:
        d =  json.load(f)
        
    d_phi      = d['dphi'] 
    d_theta    = d['dtheta'] 
    cell_areas = d['cell_area']
    lat_array  = d['lat'] 
    lon_array  = d['lon']
    #save_dict_to_json(d, "changes_lat_lon_cell-area.json", save_dir = save_dir)
    return d_phi, d_theta, cell_areas, lat_array, lon_array

In [12]:
d_phi, d_theta, cell_areas, lat_array, lon_array = read_dlon_dlat(save_dir)

['/home/hanna/images_simple_statsmodels/changes_lat_lon_cell-area.json']


In [13]:
def area_grid_cell(c_lat, d_lat, d_lon):
        """
        c_lat : float
            Centre point longitude, latitude in degrees

        d_lat, d_lon : float
            delta lat lon in degrees

        Returns : area in km^2

        cdo : If the grid cell area have to be computed it is scaled with the earth radius to square meters.
        """
        R = 6371000  # in M
        # area er egentlig R**2
        area = R*(np.sin((c_lat + d_lat)*np.pi/180) - np.sin((c_lat - d_lat)*np.pi/180) )*(2*d_lon*np.pi/180) # R**2
        return np.abs(area)


In [21]:
lats = np.arange(30, 56, 0.25) # numpy split these.
lons = np.arange(-15, 30, 0.25) # numpy split these 

c_lat = np.array(lat_array)
c_lon = np.array(lon_array) 
d_theta = np.array(d_theta) 
d_phi = np.array(d_phi)

write_fil = True
era_step = 0.25

data = {}
counter = 0
for lat in lats:
    data[str(lat)] = {}
    counter+=1
    for lon in lons:
        data[str(lat)][str(lon)] = {}
        era_AREA = area_grid_cell(lat, era_step/2, era_step/2)
        print("lat = {}, lon = {}".format(lat, lon))
        era_up    = lat + era_step/2
        era_down  = lat - era_step/2
        era_left  = lon - era_step/2
        era_right = lon + era_step/2

        # to ensure correct sign
        cmk_left  = c_lon - np.abs(d_phi)   #- era_right
        cmk_right = c_lon + np.abs(d_phi)   #- era_left

        # TODO : Sjekk d theta og lignende.
        cmk_up    = c_lat + np.abs(d_theta) #- era_down
        cmk_down  = c_lat - np.abs(d_theta) #- era_up

        data[str(lat)][str(lon)]['centre'] = {}
        data[str(lat)][str(lon)]['down'] = {}
        data[str(lat)][str(lon)]['up'] = {}
        data[str(lat)][str(lon)]['right'] = {}
        data[str(lat)][str(lon)]['left'] = {}
        data[str(lat)][str(lon)]['corner'] = {}

        idx_left_boundary  = np.intersect1d(np.argwhere(cmk_right > era_left),  np.argwhere(cmk_left < era_left) )
        idx_right_boundary = np.intersect1d(np.argwhere(cmk_right > era_right), np.argwhere(cmk_left < era_right) )
        idx_up_boundary    = np.intersect1d(np.argwhere(cmk_up > era_up),       np.argwhere(cmk_down < era_up) )
        idx_down_boundary  = np.intersect1d(np.argwhere(cmk_up > era_down),     np.argwhere(cmk_down < era_down) )

        #data[str(lat)][str(lon)]['down']['index']  = idx_down_boundary.reshape(-1).tolist()
        #data[str(lat)][str(lon)]['up']['index']    = idx_up_boundary.reshape(-1).tolist()
        #data[str(lat)][str(lon)]['left']['index']  = idx_left_boundary.reshape(-1).tolist()
        #data[str(lat)][str(lon)]['right']['index'] = idx_right_boundary.reshape(-1).tolist()

        idx_lower_right_corner = np.intersect1d(idx_down_boundary, idx_right_boundary)
        idx_lower_left_corner  = np.intersect1d(idx_down_boundary, idx_left_boundary)
        idx_upper_left_corner  = np.intersect1d(idx_up_boundary, idx_left_boundary)
        idx_upper_right_corner = np.intersect1d(idx_up_boundary, idx_right_boundary)

        corner_idx         = np.concatenate([idx_lower_right_corner, idx_lower_left_corner,
                                             idx_upper_left_corner, idx_upper_right_corner]) # corner idx
        data[str(lat)][str(lon)]['corner']['index'] = corner_idx.reshape(-1).tolist()
        #assert len(corner_idx) == 4, "Not correct number of corners, nbr = {}.".format(len(corner_idx))

        # TODO add new centre corners. And calculate  
        llc_dlat = subtract(cmk_up[idx_lower_left_corner], era_down)/2
        llc_dlon = subtract(era_left, cmk_right[idx_lower_left_corner])/2
        llc_lat  = era_down + llc_dlat

        llc_area = area_grid_cell(llc_lat, llc_dlat, llc_dlon)
        #print("area lower left area {}".format(f/era_AREA))

        lrc_dlat = subtract(cmk_up[idx_lower_right_corner], era_down)/2
        lrc_dlon = subtract(cmk_left[idx_lower_right_corner], era_right)/2

        lrc_lat  = era_down + lrc_dlat
        lrc_area = area_grid_cell(lrc_lat, lrc_dlat, lrc_dlon)
        #print("area lower right area {}".format(h/era_AREA))

        ulc_dlat = subtract(era_up, cmk_down[idx_upper_left_corner])/2
        ulc_dlon = subtract(era_left, cmk_right[idx_upper_left_corner])/2
        ulc_lat = era_up - ulc_dlat
        ulc_area = area_grid_cell(ulc_lat, ulc_dlat, ulc_dlon)
        #print("area upper left area {}".format(i/era_AREA))

        urc_dlat = subtract(era_up, cmk_down[idx_upper_right_corner])/2
        urc_dlon = subtract(cmk_left[idx_upper_right_corner], era_right)/2
        urc_lat  = era_up - ulc_dlat
        urc_area = area_grid_cell(urc_lat, urc_dlat, urc_dlon)
        #print("area upper right area {}".format(j/era_AREA))
        #print("SUM corners {}".format((i+j+h+f)/era_AREA))

        corner_areas = np.array([lrc_area, llc_area, ulc_area, urc_area])
        data[str(lat)][str(lon)]['corner']['area'] = np.concatenate(corner_areas).reshape(-1).tolist()

        #assert len(corner_idx) == 4, "Expected four corners, got {}".format(len(corner_idx))

        #max_lon, min_lon = np.max(c_lon[corner_idx]), np.min(c_lon[corner_idx])
        #max_lat, min_lat = np.max(c_lat[corner_idx]), np.min(c_lat[corner_idx])

        # Removes Corners
        # TODO test if this removes indecies. 
        idx_down_boundary = np.array(idx_down_boundary)[np.isin(idx_down_boundary, corner_idx, invert = True)]
        idx_up_boundary = np.array(idx_up_boundary)[np.isin(idx_up_boundary, corner_idx, invert = True)]
        idx_left_boundary = np.array(idx_left_boundary)[np.isin(idx_left_boundary, corner_idx, invert = True)]
        idx_right_boundary = np.array(idx_right_boundary)[np.isin(idx_right_boundary, corner_idx, invert = True)]    
        #idx_down_boundary = np.array(idx_down_boundary)[np.isin(idx_down_boundary, corner_idx invert = True)]
                       
        # subsection left boundary OLD
        low_bound = np.argwhere(cmk_down[idx_left_boundary] < era_up  )
        up_bound  = np.argwhere(cmk_up[idx_left_boundary]   > era_down )
        sub_section_left = np.intersect1d(low_bound, up_bound)

        
        idx_l = np.array(idx_left_boundary)[np.array(sub_section_left)]
        data[str(lat)][str(lon)]['left']['index'] = idx_l[np.isin(idx_l, corner_idx, invert = True)].tolist()

        # subsection right boundary
        low_bound = np.argwhere( cmk_down[idx_right_boundary] < era_up )
        up_bound  = np.argwhere( cmk_up[idx_right_boundary] > era_down)
        sub_section_right = np.intersect1d(low_bound, up_bound)
        
        idx_r = np.array(idx_right_boundary)[np.array(sub_section_right)]
        data[str(lat)][str(lon)]['right']['index'] = idx_r[np.isin(idx_r, corner_idx, invert = True)].tolist()
        #sub_section_right.reshape(-1).tolist()

        # Subsection Down Boundary
        one = np.argwhere(cmk_left[idx_down_boundary]  > era_left)
        two = np.argwhere(cmk_right[idx_down_boundary] < era_right)
        sub_section_down = np.intersect1d(one, two)
        idx_d = np.array(idx_down_boundary)[np.array(sub_section_down)]
        
        data[str(lat)][str(lon)]['down']['index'] =  idx_d[np.isin(idx_d, corner_idx, invert = True)].tolist()
        #sub_section_down.reshape(-1).tolist()

        # subsection up boundary
        one = np.argwhere( cmk_left[idx_up_boundary]  > era_left)
        two = np.argwhere( cmk_right[idx_up_boundary] < era_right)
        sub_section_up = np.intersect1d(one, two)
        idx_u = np.array(idx_up_boundary)[np.array(sub_section_up)]
        data[str(lat)][str(lon)]['up']['index'] = idx_u[np.isin(idx_u, corner_idx, invert = True)].tolist()
        #sub_section_up.reshape(-1).tolist()

        # test that these are empty
        t1 = np.intersect1d(idx_d, idx_u)
        t2 = np.intersect1d(idx_r, idx_l)
        # assert len(t1) == len(t2) == 0, "intercept up down  {}, intersect left right {}.".format(t1, t2)

        if len(t1) != 0:
            print("\n Problem up, down lat {} lon {}. \n".format(lat, lon))


        if len(t2) != 0:
            print("\n Problem left, right lat {} lon {}. \n".format(lat, lon))
        # Calculate Boundaries

        # AREA left boundary
        dlon_lf = subtract(cmk_right[idx_left_boundary][sub_section_left], era_left)/2 
        dlat_lf = d_theta[idx_left_boundary][sub_section_left]
        lat_lf  = c_lat[idx_left_boundary][sub_section_left]
        #lon_lf  = c_lon[idx_left_boundary][sub_section_left]
        
        left_areas = area_grid_cell(lat_lf, dlat_lf, np.abs(dlon_lf))
        data[str(lat)][str(lon)]['left']['area'] = left_areas.reshape(-1).tolist()

        a = np.sum(left_areas)

        #fraction_left = (cloud_mask_array[idx_left_boundary][sub_section_left]*area_grid_cell(lat_lf,
        #                                                                                      dlat_lf,                                                                                  
        #                                                                                      dlon_lf)/era_AREA).sum()
        #data[str(lat)][str(lon)]['left']['fraction'] = fraction_left.tolist()   
        # AREA right boundary
        dlon_rb = subtract(era_right, cmk_left[idx_right_boundary][sub_section_right])/2
        dlat_rb = d_theta[idx_right_boundary][sub_section_right]
        lat_rb  = c_lat[idx_right_boundary][sub_section_right]
        #lon_rb = c_lon[idx_right_boundary][sub_section_right]
        right_area = area_grid_cell(lat_rb, dlat_rb, np.abs(dlon_rb)) 
        data[str(lat)][str(lon)]['right']['area'] = right_area.ravel().tolist()
        b = np.sum(right_area)

        #fraction_right = (cloud_mask_array[idx_right_boundary][sub_section_right]*area_grid_cell(lat_rb,
        #                                                                                        dlat_rb,
        #                                                                                         dlon_rb)/era_AREA).sum()
        #data[str(lat)][str(lon)]['right']['fraction'] = fraction_right.tolist()
        # AREA down boundary
        dlat_down = subtract(era_down, cmk_up[idx_down_boundary][sub_section_down])/2
        lat_down = era_down + dlat_down
        dlon_down = d_phi[idx_down_boundary][sub_section_down]
        #lon_down =  c_lon[idx_down_boundary][sub_section_down]
        down_area = area_grid_cell(lat_down, dlat_down, dlon_down)
        c = np.sum(down_area)
        data[str(lat)][str(lon)]['down']['area'] = down_area.reshape(-1).tolist()
        #fraction_down = (cloud_mask_array[idx_down_boundary][sub_section_down]*area_grid_cell(lat_down,
        #                                                                                      dlat_down,
        #                                                                                      dlon_down)/era_AREA).sum()
        #data[str(lat)][str(lon)]['down']['fraction'] = fraction_down.tolist()
        # AREA up
        dlat_up = subtract(era_up, cmk_down[idx_up_boundary][sub_section_up])/2
        lat_up = era_up - dlat_up
        #lon_up = c_lon[idx_up_boundary][sub_section_up]
        dlon_up = d_phi[idx_up_boundary][sub_section_up]
        up_area = area_grid_cell(lat_up, dlat_up, np.abs(dlon_up))
        data[str(lat)][str(lon)]['up']['area'] = up_area.reshape(-1).tolist()
        d = np.sum(up_area)

        #fraction_up = (cloud_mask_array[idx_up_boundary][sub_section_up]*area_grid_cell(lat_up,
        #                                                                                 dlat_up,
        #                                                                                 dlon_up)/era_AREA).sum()
        #data[str(lat)][str(lon)]['up']['fraction'] = fraction_up.tolist()
        # Index centres. Can safly assume centre cells are correct.
        idx_centre_one = np.intersect1d(np.argwhere(cmk_left  > era_left),
                                        np.argwhere(cmk_right < era_right))


        idx_centre_two = np.intersect1d(np.argwhere(cmk_up   < era_up),
                                        np.argwhere(cmk_down >  era_down))

        idx_centre_cells = np.intersect1d( idx_centre_one, idx_centre_two )
        data[str(lat)][str(lon)]['centre']['index'] = idx_centre_cells.ravel().tolist()

        lat_centre_cells = c_lat[idx_centre_cells]
        dlat_centre      = d_theta[idx_centre_cells]
        dlon_centre      = d_phi[idx_centre_cells]
        
        centre_area      = area_grid_cell(lat_centre_cells, dlat_centre, dlon_centre)

        data[str(lat)][str(lon)]['centre']['area'] = centre_area.reshape(-1).tolist()

    #print("Saves file every lat {} lon {}.".format(lat, lon))
    filnavn = 'ERA5_grid_resample_includes_changes_to_indexes_from_{}_{}.json'.format(float(lat)-5, lat)
    if lat%5==0 and lat > 30:
        filnavn = 'ERA5_grid_resample_includes_changes_to_indexes_from_{}_{}.json'.format(float(lat)-5, lat)
        with open(save_dir + filnavn, 'w') as f:
            json.dump(data, f)
        data = {}

#filnavn = 'ERA5_grid_resample_includes_lat{}_lon{}.json'.format(lat, lon)
#with open(save_dir + filnavn, 'w') as f:
#    json.dump(data[str(lat)][str(lon)], f)
#data = {}        


lat = 30.0, lon = -15.0


/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in greater
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in greater
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in less
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in greater
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in less
/home/hanna/anaconda3/envs/sciclouds/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in greater
/

lat = 30.0, lon = -14.75
lat = 30.0, lon = -14.5
lat = 30.0, lon = -14.25
lat = 30.0, lon = -14.0
lat = 30.0, lon = -13.75
lat = 30.0, lon = -13.5
lat = 30.0, lon = -13.25
lat = 30.0, lon = -13.0
lat = 30.0, lon = -12.75
lat = 30.0, lon = -12.5
lat = 30.0, lon = -12.25
lat = 30.0, lon = -12.0
lat = 30.0, lon = -11.75
lat = 30.0, lon = -11.5
lat = 30.0, lon = -11.25
lat = 30.0, lon = -11.0
lat = 30.0, lon = -10.75
lat = 30.0, lon = -10.5
lat = 30.0, lon = -10.25
lat = 30.0, lon = -10.0
lat = 30.0, lon = -9.75
lat = 30.0, lon = -9.5
lat = 30.0, lon = -9.25
lat = 30.0, lon = -9.0
lat = 30.0, lon = -8.75
lat = 30.0, lon = -8.5
lat = 30.0, lon = -8.25
lat = 30.0, lon = -8.0
lat = 30.0, lon = -7.75
lat = 30.0, lon = -7.5
lat = 30.0, lon = -7.25
lat = 30.0, lon = -7.0
lat = 30.0, lon = -6.75
lat = 30.0, lon = -6.5
lat = 30.0, lon = -6.25
lat = 30.0, lon = -6.0
lat = 30.0, lon = -5.75
lat = 30.0, lon = -5.5
lat = 30.0, lon = -5.25
lat = 30.0, lon = -5.0
lat = 30.0, lon = -4.75
lat = 30.0, lon 

lat = 30.25, lon = 26.25
lat = 30.25, lon = 26.5
lat = 30.25, lon = 26.75
lat = 30.25, lon = 27.0
lat = 30.25, lon = 27.25
lat = 30.25, lon = 27.5
lat = 30.25, lon = 27.75
lat = 30.25, lon = 28.0
lat = 30.25, lon = 28.25
lat = 30.25, lon = 28.5
lat = 30.25, lon = 28.75
lat = 30.25, lon = 29.0
lat = 30.25, lon = 29.25
lat = 30.25, lon = 29.5
lat = 30.25, lon = 29.75
lat = 30.5, lon = -15.0
lat = 30.5, lon = -14.75
lat = 30.5, lon = -14.5
lat = 30.5, lon = -14.25
lat = 30.5, lon = -14.0
lat = 30.5, lon = -13.75
lat = 30.5, lon = -13.5
lat = 30.5, lon = -13.25
lat = 30.5, lon = -13.0
lat = 30.5, lon = -12.75
lat = 30.5, lon = -12.5
lat = 30.5, lon = -12.25
lat = 30.5, lon = -12.0
lat = 30.5, lon = -11.75
lat = 30.5, lon = -11.5
lat = 30.5, lon = -11.25
lat = 30.5, lon = -11.0
lat = 30.5, lon = -10.75
lat = 30.5, lon = -10.5
lat = 30.5, lon = -10.25
lat = 30.5, lon = -10.0
lat = 30.5, lon = -9.75
lat = 30.5, lon = -9.5
lat = 30.5, lon = -9.25
lat = 30.5, lon = -9.0
lat = 30.5, lon = -8.75


lat = 30.75, lon = 22.25
lat = 30.75, lon = 22.5
lat = 30.75, lon = 22.75
lat = 30.75, lon = 23.0
lat = 30.75, lon = 23.25
lat = 30.75, lon = 23.5
lat = 30.75, lon = 23.75
lat = 30.75, lon = 24.0
lat = 30.75, lon = 24.25
lat = 30.75, lon = 24.5
lat = 30.75, lon = 24.75
lat = 30.75, lon = 25.0
lat = 30.75, lon = 25.25
lat = 30.75, lon = 25.5
lat = 30.75, lon = 25.75
lat = 30.75, lon = 26.0
lat = 30.75, lon = 26.25
lat = 30.75, lon = 26.5
lat = 30.75, lon = 26.75
lat = 30.75, lon = 27.0
lat = 30.75, lon = 27.25
lat = 30.75, lon = 27.5
lat = 30.75, lon = 27.75
lat = 30.75, lon = 28.0
lat = 30.75, lon = 28.25
lat = 30.75, lon = 28.5
lat = 30.75, lon = 28.75
lat = 30.75, lon = 29.0
lat = 30.75, lon = 29.25
lat = 30.75, lon = 29.5
lat = 30.75, lon = 29.75
lat = 31.0, lon = -15.0
lat = 31.0, lon = -14.75
lat = 31.0, lon = -14.5
lat = 31.0, lon = -14.25
lat = 31.0, lon = -14.0
lat = 31.0, lon = -13.75
lat = 31.0, lon = -13.5
lat = 31.0, lon = -13.25
lat = 31.0, lon = -13.0
lat = 31.0, lon = -1

...

lat = 53.75, lon = -1.75
lat = 53.75, lon = -1.5
lat = 53.75, lon = -1.25
lat = 53.75, lon = -1.0
lat = 53.75, lon = -0.75
lat = 53.75, lon = -0.5
lat = 53.75, lon = -0.25
lat = 53.75, lon = 0.0
lat = 53.75, lon = 0.25
lat = 53.75, lon = 0.5
lat = 53.75, lon = 0.75
lat = 53.75, lon = 1.0
lat = 53.75, lon = 1.25
lat = 53.75, lon = 1.5
lat = 53.75, lon = 1.75
lat = 53.75, lon = 2.0
lat = 53.75, lon = 2.25
lat = 53.75, lon = 2.5
lat = 53.75, lon = 2.75
lat = 53.75, lon = 3.0
lat = 53.75, lon = 3.25
lat = 53.75, lon = 3.5
lat = 53.75, lon = 3.75
lat = 53.75, lon = 4.0
lat = 53.75, lon = 4.25
lat = 53.75, lon = 4.5
lat = 53.75, lon = 4.75
lat = 53.75, lon = 5.0
lat = 53.75, lon = 5.25
lat = 53.75, lon = 5.5
lat = 53.75, lon = 5.75
lat = 53.75, lon = 6.0
lat = 53.75, lon = 6.25
lat = 53.75, lon = 6.5
lat = 53.75, lon = 6.75
lat = 53.75, lon = 7.0
lat = 53.75, lon = 7.25
lat = 53.75, lon = 7.5
lat = 53.75, lon = 7.75
lat = 53.75, lon = 8.0
lat = 53.75, lon = 8.25
lat = 53.75, lon = 8.5
lat = 

lat = 54.25, lon = -5.75
lat = 54.25, lon = -5.5
lat = 54.25, lon = -5.25
lat = 54.25, lon = -5.0
lat = 54.25, lon = -4.75
lat = 54.25, lon = -4.5
lat = 54.25, lon = -4.25
lat = 54.25, lon = -4.0
lat = 54.25, lon = -3.75
lat = 54.25, lon = -3.5
lat = 54.25, lon = -3.25
lat = 54.25, lon = -3.0
lat = 54.25, lon = -2.75
lat = 54.25, lon = -2.5
lat = 54.25, lon = -2.25
lat = 54.25, lon = -2.0
lat = 54.25, lon = -1.75
lat = 54.25, lon = -1.5
lat = 54.25, lon = -1.25
lat = 54.25, lon = -1.0
lat = 54.25, lon = -0.75
lat = 54.25, lon = -0.5
lat = 54.25, lon = -0.25
lat = 54.25, lon = 0.0
lat = 54.25, lon = 0.25
lat = 54.25, lon = 0.5
lat = 54.25, lon = 0.75
lat = 54.25, lon = 1.0
lat = 54.25, lon = 1.25
lat = 54.25, lon = 1.5
lat = 54.25, lon = 1.75
lat = 54.25, lon = 2.0
lat = 54.25, lon = 2.25
lat = 54.25, lon = 2.5
lat = 54.25, lon = 2.75
lat = 54.25, lon = 3.0
lat = 54.25, lon = 3.25
lat = 54.25, lon = 3.5
lat = 54.25, lon = 3.75
lat = 54.25, lon = 4.0
lat = 54.25, lon = 4.25
lat = 54.25, 

lat = 54.75, lon = -9.75
lat = 54.75, lon = -9.5
lat = 54.75, lon = -9.25
lat = 54.75, lon = -9.0
lat = 54.75, lon = -8.75
lat = 54.75, lon = -8.5
lat = 54.75, lon = -8.25
lat = 54.75, lon = -8.0
lat = 54.75, lon = -7.75
lat = 54.75, lon = -7.5
lat = 54.75, lon = -7.25
lat = 54.75, lon = -7.0
lat = 54.75, lon = -6.75
lat = 54.75, lon = -6.5
lat = 54.75, lon = -6.25
lat = 54.75, lon = -6.0
lat = 54.75, lon = -5.75
lat = 54.75, lon = -5.5
lat = 54.75, lon = -5.25
lat = 54.75, lon = -5.0
lat = 54.75, lon = -4.75
lat = 54.75, lon = -4.5
lat = 54.75, lon = -4.25
lat = 54.75, lon = -4.0
lat = 54.75, lon = -3.75
lat = 54.75, lon = -3.5
lat = 54.75, lon = -3.25
lat = 54.75, lon = -3.0
lat = 54.75, lon = -2.75
lat = 54.75, lon = -2.5
lat = 54.75, lon = -2.25
lat = 54.75, lon = -2.0
lat = 54.75, lon = -1.75
lat = 54.75, lon = -1.5
lat = 54.75, lon = -1.25
lat = 54.75, lon = -1.0
lat = 54.75, lon = -0.75
lat = 54.75, lon = -0.5
lat = 54.75, lon = -0.25
lat = 54.75, lon = 0.0
lat = 54.75, lon = 0.

lat = 55.25, lon = -13.75
lat = 55.25, lon = -13.5
lat = 55.25, lon = -13.25
lat = 55.25, lon = -13.0
lat = 55.25, lon = -12.75
lat = 55.25, lon = -12.5
lat = 55.25, lon = -12.25
lat = 55.25, lon = -12.0
lat = 55.25, lon = -11.75
lat = 55.25, lon = -11.5
lat = 55.25, lon = -11.25
lat = 55.25, lon = -11.0
lat = 55.25, lon = -10.75
lat = 55.25, lon = -10.5
lat = 55.25, lon = -10.25
lat = 55.25, lon = -10.0
lat = 55.25, lon = -9.75
lat = 55.25, lon = -9.5
lat = 55.25, lon = -9.25
lat = 55.25, lon = -9.0
lat = 55.25, lon = -8.75
lat = 55.25, lon = -8.5
lat = 55.25, lon = -8.25
lat = 55.25, lon = -8.0
lat = 55.25, lon = -7.75
lat = 55.25, lon = -7.5
lat = 55.25, lon = -7.25
lat = 55.25, lon = -7.0
lat = 55.25, lon = -6.75
lat = 55.25, lon = -6.5
lat = 55.25, lon = -6.25
lat = 55.25, lon = -6.0
lat = 55.25, lon = -5.75
lat = 55.25, lon = -5.5
lat = 55.25, lon = -5.25
lat = 55.25, lon = -5.0
lat = 55.25, lon = -4.75
lat = 55.25, lon = -4.5
lat = 55.25, lon = -4.25
lat = 55.25, lon = -4.0
lat 

lat = 55.5, lon = 27.0
lat = 55.5, lon = 27.25
lat = 55.5, lon = 27.5
lat = 55.5, lon = 27.75
lat = 55.5, lon = 28.0
lat = 55.5, lon = 28.25
lat = 55.5, lon = 28.5
lat = 55.5, lon = 28.75
lat = 55.5, lon = 29.0
lat = 55.5, lon = 29.25
lat = 55.5, lon = 29.5
lat = 55.5, lon = 29.75
lat = 55.75, lon = -15.0
lat = 55.75, lon = -14.75
lat = 55.75, lon = -14.5
lat = 55.75, lon = -14.25
lat = 55.75, lon = -14.0
lat = 55.75, lon = -13.75
lat = 55.75, lon = -13.5
lat = 55.75, lon = -13.25
lat = 55.75, lon = -13.0
lat = 55.75, lon = -12.75
lat = 55.75, lon = -12.5
lat = 55.75, lon = -12.25
lat = 55.75, lon = -12.0
lat = 55.75, lon = -11.75
lat = 55.75, lon = -11.5
lat = 55.75, lon = -11.25
lat = 55.75, lon = -11.0
lat = 55.75, lon = -10.75
lat = 55.75, lon = -10.5
lat = 55.75, lon = -10.25
lat = 55.75, lon = -10.0
lat = 55.75, lon = -9.75
lat = 55.75, lon = -9.5
lat = 55.75, lon = -9.25
lat = 55.75, lon = -9.0
lat = 55.75, lon = -8.75
lat = 55.75, lon = -8.5
lat = 55.75, lon = -8.25
lat = 55.75

In [ ]:
print(data)

In [ ]:
filnavn = 'ERA5_grid_resample_includes_lat{}_lon{}.json'.format(lat, lon)
with open(save_dir + filnavn, 'w') as f:
    json.dump(data, f)
#data = {}

In [ ]:
data

In [ ]:
data[str(lat)][str(lon)]

In [ ]:
import simplejson as json

In [ ]:
def save(data, filnavn):
    #filnavn = 'ERA5_grid_resample_counter{}_lat{}_lon{}.json'.format(counter, lat, lon)
    with open(save_dir + filnavn, 'w') as f:
        json.dump(data, f)
#data = {}

In [ ]:
for key, item in data.items():
    for key2, item2 in item.items():
        #save(item2, "a.json")
        for key3, item3 in item2.items():
            #save(item3, "b.json")
            for key4, item4 in item3.items():
                #save(item4, "c.json")
                print(item4)

In [40]:
type(item)

dict

In [45]:
import json

In [37]:
filnavn = 'ERA5_grid_resample_counter{}_lat{}_lon{}.json'.format(counter, lat, lon)
with open(save_dir + filnavn, 'w') as f:
    json.dump(item, f)

TypeError: Object of type ndarray is not JSON serializable

In [26]:
glob.glob(save_dir + '*json')

['/home/hanna/images_simple_statsmodels/ERA_INTERIM_grid_resample_75_counter10.json',
 '/home/hanna/images_simple_statsmodels/ERA5_grid_resample_fractions_areas_res_All0.75.json',
 '/home/hanna/images_simple_statsmodels/ERA_INTERIM_grid_resample_5.json',
 '/home/hanna/images_simple_statsmodels/ERA_INTERIM_grid_resample_75_counter0.json',
 '/home/hanna/images_simple_statsmodels/changes_lat_lon_cell-area.json']

In [27]:
counter

10